### XOR 게이트

- 비선형 중 가장 심플
- 직선 하나로 데이터를 분류할 수 없다 (선형분리불가능)
- 적어도 두개의 직선이 필요함 

\begin{align}
a x_1 + b x_2 + c = 0
\end{align}

In [13]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
import tensorflow as tf


### 1. 데이터 생성

In [15]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
Y = np.array(([0],[1],[1],[0]))

### 2. Tensor Flow

#### 2-1. 모델 생성

In [17]:
x = tf.placeholder(tf.float32, shape = [None, 2])
t = tf.placeholder(tf.float32, shape = [None, 1])

#### 2-2. 입력층-은닉층

In [19]:
w = tf.Variable(tf.truncated_normal([2,2]))
b = tf.Variable(tf.zeros([2]))
h = tf.nn.sigmoid(tf.matmul(x, w) + b)

#### 2-3. 은닉층-출력층

In [20]:
v = tf.Variable(tf.truncated_normal([2,1]))
c = tf.Variable(tf.zeros([1]))
y = tf.nn.sigmoid(tf.matmul(h, v) + c)

In [22]:
cross_entropy = -tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
pred = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

#### 2-4. 모델 학습

In [23]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [26]:
for epoch in range(4000):
    sess.run(train_step, feed_dict = {x:X, t:Y})
    if epoch % 1000 == 0:
        print('epoch:',epoch)

epoch: 0
epoch: 1000
epoch: 2000
epoch: 3000


#### 2-5. 결과 확인

In [27]:
res = pred.eval(session = sess, feed_dict = {x:X, t:Y})
prob = y.eval(session = sess, feed_dict={x:X})

In [28]:
print("classified:", res)
print("probability:", prob)

classified: [[ True]
 [ True]
 [ True]
 [ True]]
probability: [[0.00855437]
 [0.99227905]
 [0.988315  ]
 [0.0072985 ]]


### 3. Keras

#### 3-1. 모델 설정

In [3]:
model = Sequential()

#### 3-2. 입력층 - 은닉층

In [4]:
model.add(Dense(input_dim = 2, units = 2))
model.add(Activation('sigmoid'))

#### 3-3. 은닉층 - 출력층

In [5]:
model.add(Dense(units = 1))
model.add(Activation('sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = SGD(lr = 0.1))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


#### 3-4. 모델 학습

In [30]:
model.fit(X,Y, epochs = 4000, batch_size = 5, verbose = False)

#### 3-5. 결과

In [32]:
classes = model.predict_classes(X, batch_size = 3)
prob = model.predict_proba(X, batch_size = 3)
print('classified:', Y == classes)
print('probability:', prob)

classified: [[ True]
 [ True]
 [ True]
 [ True]]
probability: [[0.04195528]
 [0.9717819 ]
 [0.9634061 ]
 [0.03604312]]


학습을 성공하기 위한 최소의 epochs와 batch_size를 구하는 것을 연구해야겠다